<a href="https://colab.research.google.com/github/aishafarooque/Tweet-Intimacy-Analysis/blob/main/Baseline_BERT_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetuning distilbert-base-uncased for Twitter and Reddit

In [ ]:
from IPython.display import clear_output 

! pip install --upgrade pip
! pip install transformers
! pip install tqdm
! pip install datasets
! pip install evaluate


# (Aisha) Clear output because it tends to get long. 
# But these libraries are always successfully installed.
clear_output()

In [ ]:
import numpy as np
import pandas as pd
import transformers
from datasets import Dataset,load_dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
# Remove Twitter's train.csv if it already exists
! rm -rf train.csv

# Download Twitter's training data
! wget https://raw.githubusercontent.com/aishafarooque/Tweet-Intimacy-Analysis/main/train.csv

# Rename train.csv -> twitter_train.csv for more clarity
! mv train.csv twitter_train.csv

--2022-12-05 07:03:46--  https://raw.githubusercontent.com/aishafarooque/Tweet-Intimacy-Analysis/main/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 813066 (794K) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>] 794.01K  --.-KB/s    in 0.02s   

2022-12-05 07:03:46 (51.1 MB/s) - ‘train.csv’ saved [813066/813066]



In [ ]:
# Sanitze working directory
! rm -rf /content/__MACOSX
! rm -rf /content/annotated_question_intimacy_data

# Removing the .zip file if it already exists.
! rm -rf annotated_question_intimacy_data.zip

# Download the dataset from the author's GitHub repository.
! wget https://raw.githubusercontent.com/Jiaxin%2DPei/Quantifying%2DIntimacy%2Din%2DLanguage/main/data/annotated_question_intimacy_data.zip

# Unzip the file. 
! unzip /content/annotated_question_intimacy_data.zip

--2022-12-05 07:03:47--  https://raw.githubusercontent.com/Jiaxin%2DPei/Quantifying%2DIntimacy%2Din%2DLanguage/main/data/annotated_question_intimacy_data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94741 (93K) [application/zip]
Saving to: ‘annotated_question_intimacy_data.zip’

annotated_question_ 100%[===================>]  92.52K  --.-KB/s    in 0.004s  

2022-12-05 07:03:47 (20.2 MB/s) - ‘annotated_question_intimacy_data.zip’ saved [94741/94741]

Archive:  /content/annotated_question_intimacy_data.zip
   creating: annotated_question_intimacy_data/
  inflating: annotated_question_intimacy_data/final_train.txt  
   creating: __MACOSX/
   creating: __MACOSX/annotated_question_intimacy_data/
  inflating: __MACOSX/annotated_question_intimacy_data/._fin

## Downloading data

In [ ]:
import pandas as pd

twitter_df_train = pd.read_csv('/content/twitter_train.csv', on_bad_lines='skip')
twitter_df_train = twitter_df_train.drop(columns=['language'])
twitter_df_train = twitter_df_train.rename(columns={'text': 'document', 'label': 'labels'})

print("Dataset size:", len(twitter_df_train), '\n')
twitter_df_train.info()

Dataset size: 9491 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9491 entries, 0 to 9490
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   document  9491 non-null   object 
 1   labels    9491 non-null   float64
dtypes: float64(1), object(1)
memory usage: 148.4+ KB


In [ ]:
reddit_df_train = pd.read_csv('/content/annotated_question_intimacy_data/final_train.txt', 
                              sep='\t', header=None, names=['document', 'labels'])

print("Dataset size:", len(reddit_df_train), '\n')
reddit_df_train.info()

Dataset size: 1797 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1797 entries, 0 to 1796
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   document  1797 non-null   object 
 1   labels    1797 non-null   float64
dtypes: float64(1), object(1)
memory usage: 28.2+ KB


In [ ]:
combined_df = pd.concat([twitter_df_train,reddit_df_train])
print("Dataset size:", len(combined_df), '\n')
combined_df.info()

Dataset size: 11288 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11288 entries, 0 to 1796
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   document  11288 non-null  object 
 1   labels    11288 non-null  float64
dtypes: float64(1), object(1)
memory usage: 264.6+ KB


#### Linear Mapping

In [ ]:
A, B, C, D = -1, 1, 1, 5
scale = (D-C)/(B-A)
offset = -A*(D-C)/(B-A) + C

for index, row in reddit_df_train.iterrows():
  iScore = row['labels']

  # If the cell is re-run without clearing local variables, we'll
  # double convert the values between the 1-5 range resulting in values between
  # 5-10. This condition makes sure original scores from Reddit are not already
  #  greater than 1. 
  if iScore > 1:
    break

  q = iScore * scale + offset
  reddit_df_train.at[index, 'labels'] = round(q, 1)
  
reddit_df_train.head()

,document,labels
0,What are the most mediocre animals in the anim...,2.3
1,What's the difference between an allergic reac...,3.1
2,What is your favorite subreddit that not every...,3.1
3,What's the most disgusting meal you've ever ea...,3.5
4,Whats one question you hate being asked?,4.0


In [ ]:
import random as rn
from datasets import Dataset, DatasetDict

import datasets

twitter_dataset = Dataset.from_pandas(twitter_df_train)
twitter_dataset = twitter_dataset.train_test_split(test_size=0.2)
twitter_dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'labels'],
        num_rows: 7592
    })
    test: Dataset({
        features: ['document', 'labels'],
        num_rows: 1899
    })
})

In [ ]:
reddit_dataset = Dataset.from_pandas(reddit_df_train)
reddit_dataset = reddit_dataset.train_test_split(test_size=0.2)
reddit_dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'labels'],
        num_rows: 1437
    })
    test: Dataset({
        features: ['document', 'labels'],
        num_rows: 360
    })
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
tokenizer('USA will win the world cup')['input_ids']

[101, 3915, 2097, 2663, 1996, 2088, 2452, 102]

In [ ]:
tokenized_sample = []

for i in tokenizer('USA will win the world cup')['input_ids']:
  tokenized_sample.append(tokenizer.decode(i))

tokenized_sample

['[CLS]', 'usa', 'will', 'win', 'the', 'world', 'cup', '[SEP]']

## Data Preprocessing

#### Tokenize Twitter Data

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["document"], padding="max_length", truncation=True)
twitter_tokenized_datasets = twitter_dataset.map(tokenize_function, batched=True)

twitter_tokenized_datasets_train = twitter_tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
twitter_tokenized_datasets_test = twitter_tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
twitter_tokenized_datasets_train

Dataset({
    features: ['document', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 1000
})

In [ ]:
twitter_tokenized_datasets_test

Dataset({
    features: ['document', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 1000
})

#### Tokenize Reddit Data

In [ ]:
reddit_tokenized_datasets = reddit_dataset.map(tokenize_function, batched=True)
reddit_tokenized_datasets

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['document', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 1437
    })
    test: Dataset({
        features: ['document', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 360
    })
})

## BERT Base Uncased for Sequence Classification

#### Model Configuration

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=1)


Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [ ]:
model.resize_token_embeddings(len(tokenizer))


Embedding(30522, 768, padding_idx=0)

#### Training only using Twitter

In [ ]:

metric_name = "pearson"
metric = load_metric('glue', 'stsb')
metric 



Metric(name: "glue", features: {'predictions': Value(dtype='float32', id=None), 'references': Value(dtype='float32', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = datasets.load_metric('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print

In [ ]:
from datasets import load_metric
from sklearn.metrics import mean_squared_error
from scipy import stats

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)


In [ ]:
fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)


{'pearson': 0.14462158210542375, 'spearmanr': 0.14462158210542372}

In [ ]:
from transformers import TrainingArguments, Trainer

OUTPUT_DIR = 'twitter_trainer'
training_args = TrainingArguments(output_dir=OUTPUT_DIR,
                                  logging_strategy="epoch",
                                  evaluation_strategy="epoch",
                                  per_device_train_batch_size=8,
                                  per_device_eval_batch_size=8,
                                  num_train_epochs=3,
                                  save_total_limit = 2,
                                  save_strategy = 'no',
                                  load_best_model_at_end=False,
                                  metric_for_best_model="pearson",
                                  )

twitter_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=twitter_tokenized_datasets_train,
    eval_dataset=twitter_tokenized_datasets_test,
    compute_metrics=compute_metrics
)

twitter_trainer.compute_metrics = compute_metrics
twitter_trainer.train()


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: document. If document are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total tr

Epoch,Training Loss,Validation Loss,Pearson,Spearmanr
1,0.215500,0.724226,0.412502,0.433367
2,0.135100,0.806192,0.391574,0.409468
3,0.084800,0.769739,0.409009,0.424968


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: document. If document are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: document. If document are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: document. If document are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *

TrainOutput(global_step=375, training_loss=0.14510909525553387, metrics={'train_runtime': 196.6766, 'train_samples_per_second': 15.253, 'train_steps_per_second': 1.907, 'total_flos': 397395108864000.0, 'train_loss': 0.14510909525553387, 'epoch': 3.0})

In [ ]:
twitter_trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: document. If document are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


{'eval_loss': 0.7697387337684631,
 'eval_pearson': 0.40900907219256627,
 'eval_spearmanr': 0.42496835028286695,
 'eval_runtime': 17.4996,
 'eval_samples_per_second': 57.144,
 'eval_steps_per_second': 7.143,
 'epoch': 3.0}

In [ ]:
from transformers import Trainer

trainer = Trainer(model=model)

def tokenize_function(examples):
    return tokenizer(examples["document"], padding="max_length", truncation=True) 

def pipeline_prediction(text):
    df=pd.DataFrame({'document':[text]})
    dataset = Dataset.from_pandas(df,preserve_index=False) 
    tokenized_datasets = dataset.map(tokenize_function)
    raw_pred, _, _ = trainer.predict(tokenized_datasets) 
    return(raw_pred[0][0])

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
pipeline_prediction("What's on your bucket list?")

  0%|          | 0/1 [00:00<?, ?ex/s]

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: document. If document are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1
  Batch size = 8


1.9304324

In [ ]:
pipeline_prediction("Where did you get married?")

  0%|          | 0/1 [00:00<?, ?ex/s]

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: document. If document are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1
  Batch size = 8


3.7036734

#### Training using Twitter train and Reddit test

In [ ]:
from transformers import TrainingArguments, Trainer

twitter_reddit_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=twitter_tokenized_datasets_train,
    eval_dataset=reddit_tokenized_datasets['train'],
    compute_metrics=compute_metrics
)

twitter_reddit_trainer.train()


The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: document. If document are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 375
  Number of trainable parameters = 66954241


Epoch,Training Loss,Validation Loss,Pearson,Spearmanr
1,0.108400,0.182191,0.962830,0.972531
2,0.076100,0.285813,0.947400,0.961618
3,0.051100,0.272361,0.951330,0.962483


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: document. If document are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1437
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: document. If document are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1437
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: document. If document are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *

TrainOutput(global_step=375, training_loss=0.07854781087239583, metrics={'train_runtime': 219.8952, 'train_samples_per_second': 13.643, 'train_steps_per_second': 1.705, 'total_flos': 397395108864000.0, 'train_loss': 0.07854781087239583, 'epoch': 3.0})

In [ ]:
twitter_reddit_trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: document. If document are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1437
  Batch size = 8


{'eval_loss': 0.2723606526851654,
 'eval_pearson': 0.9513302955418135,
 'eval_spearmanr': 0.9624829753371851,
 'eval_runtime': 25.2409,
 'eval_samples_per_second': 56.931,
 'eval_steps_per_second': 7.131,
 'epoch': 3.0}

In [ ]:
from transformers import Trainer

trainer = Trainer(model=model)
twitter_reddit_trainer_prediction_1 = pipeline_prediction("What's on your bucket list?")
twitter_reddit_trainer_prediction_1

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


  0%|          | 0/1 [00:00<?, ?ex/s]

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: document. If document are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1
  Batch size = 8


1.8093481

In [ ]:
twitter_reddit_trainer_prediction_2 = pipeline_prediction("Where did you get married?")
twitter_reddit_trainer_prediction_2

  0%|          | 0/1 [00:00<?, ?ex/s]

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: document. If document are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1
  Batch size = 8


3.5103896

#### Training using Reddit train and Twitter test

STS-B (Semantic Textual Similarity Benchmark) Determine the similarity of two sentences with a score from 1 to 5.

In [ ]:
from transformers import TrainingArguments, Trainer

reddit_twitter_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=reddit_tokenized_datasets['train'],
    eval_dataset=twitter_tokenized_datasets_train,
    compute_metrics=compute_metrics
)

reddit_twitter_trainer.compute_metrics = compute_metrics
reddit_twitter_trainer.train()


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: document. If document are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 1437
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 540
  Number of trainable parameters = 66954241


Epoch,Training Loss,Validation Loss,Pearson,Spearmanr
1,0.028800,0.925028,0.779328,0.801089
2,0.019700,0.814385,0.787937,0.806806
3,0.022700,0.818589,0.785322,0.799982


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: document. If document are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: document. If document are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: document. If document are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *

TrainOutput(global_step=540, training_loss=0.023718603452046714, metrics={'train_runtime': 261.5605, 'train_samples_per_second': 16.482, 'train_steps_per_second': 2.065, 'total_flos': 571056771437568.0, 'train_loss': 0.023718603452046714, 'epoch': 3.0})

In [ ]:
reddit_twitter_trainer.evaluate()


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: document. If document are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


{'eval_loss': 0.8185886144638062,
 'eval_pearson': 0.7853215695759056,
 'eval_spearmanr': 0.7999821929798833,
 'eval_runtime': 17.8743,
 'eval_samples_per_second': 55.946,
 'eval_steps_per_second': 6.993,
 'epoch': 3.0}

In [ ]:
from transformers import Trainer

trainer = Trainer(model=model)
reddit_twitter_trainer_prediction_1 = pipeline_prediction("What's on your bucket list?")
reddit_twitter_trainer_prediction_1

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


  0%|          | 0/1 [00:00<?, ?ex/s]

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: document. If document are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1
  Batch size = 8


1.8093481

In [ ]:
reddit_twitter_trainer_prediction_2 = pipeline_prediction("Where did you get married?")
reddit_twitter_trainer_prediction_2

  0%|          | 0/1 [00:00<?, ?ex/s]

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: document. If document are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1
  Batch size = 8


3.5103896